In [ ]:
#code for applying image-processing techniques
import streamlit as st

from PIL import Image

import numpy as np

import cv2

from skimage import exposure, morphology, filters

from skimage.feature import canny

from skimage.filters import sobel, prewitt

from skimage import io, color

from skimage.morphology import binary_opening, binary_closing, binary_erosion, binary_dilation

import tensorflow as tf







def perform_histogram_equalization(image):

    img_array = exposure.equalize_hist(image)

    return img_array





def perform_edge_detection(image):

    edges = filters.sobel(image)

    return edges



def perform_prewitt_edge_detection(image):

    edges = prewitt(image)

    return edges



def perform_canny_edge_detection(image, sigma=1.0, low_threshold=0.1, high_threshold=0.2):

    edges = canny(image, sigma=sigma, low_threshold=low_threshold, high_threshold=high_threshold)

    return edges



# def perform_binary_opening(image):

#     opened_image = binary_opening(image)

#     return opened_image



# def perform_binary_closing(image):

#     closed_image = binary_closing(image)

#     return closed_image



# def perform_binary_erosion(image):

#     eroded_image = binary_erosion(image)

#     return eroded_image



# def perform_binary_dilation(image):

#     dilated_image = binary_dilation(image)

#     return dilated_image



def perform_image_filtering(image):

    filtered_image = cv2.GaussianBlur(image, (5, 5), 0)

    return filtered_image



# Sidebar navigation

rad = st.sidebar.selectbox("Navigation", ["Home", "Histogram Equalization", "Edge Detection", "Morphological Operations", "Image Filtering","Form Detection"])



# HOME PAGE

def main():

    uploaded_image = None

    uploaded_image = st.sidebar.file_uploader('Choose an image...', type=['jpg', 'png', 'jpeg'])

    kernel = np.ones((4,4),np.uint8)



    if uploaded_image is not None:

        img = Image.open(uploaded_image)

        img_array = np.array(img)

        img_gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)

        threshold_value = 0.3 # You may adjust this threshold value

        binary_image = cv2.threshold(img_gray,180,255,2)

        if rad == "Home":

            st.title("Classifier")

            st.image(uploaded_image, caption='Uploaded Image')





        # HISTOGRAM EQUALIZATION

        if rad == "Histogram Equalization":

            st.header("Histogram Equalization")

            processed_image = perform_histogram_equalization(img_array)

            st.image(processed_image, caption='Histogram Equalization', use_column_width=True)



        # EDGE DETECTION

        elif rad == "Edge Detection":

            st.header("Edge Detection")

            # img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)



            with st.expander('Sobel'):

                processed_image = perform_edge_detection(img_gray)

                st.image(processed_image, caption='Sobel Edge Detection', use_column_width=True)

            with st.expander('Prewitt'):

                processed_image = perform_prewitt_edge_detection(img_gray)

                st.image(processed_image, caption='Prewitt Edge Detection', use_column_width=True)

            with st.expander('Canny'):

                processed_image = perform_canny_edge_detection(img_gray)

                st.image(processed_image, caption='Canny Edge Detection', use_column_width=True)



        # MORPHOLOGICAL OPERATIONS

        elif rad == "Morphological Operations":

            st.header("Morphological Operations")

            img_gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)

            threshold_value = 0.3  # You may adjust this threshold value

            _, binary_image = cv2.threshold(img_gray, 180, 255, cv2.THRESH_BINARY)

            binary_image = binary_image.astype(np.uint8)



            with st.expander('Opening'):

                processed_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel, iterations=1)

                st.image(processed_image, caption='Morphological Image Opening', use_column_width=True)

            with st.expander('Closing'):

                processed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel, iterations=1)

                st.image(processed_image, caption='Morphological Image Closing', use_column_width=True)

            with st.expander('Erosion'):

                processed_image = cv2.erode(binary_image, kernel, iterations=1)

                st.image(processed_image, caption='Morphological Image Erosion', use_column_width=True)

            with st.expander('Dilation'):

                processed_image = cv2.dilate(binary_image, kernel, iterations=1)

                st.image(processed_image, caption='Morphological Image Dilation', use_column_width=True)



        # IMAGE FILTERING

        elif rad == "Image Filtering":

            st.header("Image Filtering")

            processed_image = perform_image_filtering(img_array)

            st.image(processed_image, caption='Image Filtering', use_column_width=True)





if _name_ == "_main_":

    main()

In [ ]:
#code for classification
import os

import cv2

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

import joblib



import cv2



# Function for histogram equalization

def histogram_equalization(image):

    equ = cv2.equalizeHist(image)

    return equ



# Function to load and preprocess images from a folder and convert them into text-based representations

def load_and_preprocess_images_to_text(folder_path, label, target_size=(64, 64)):

    images_text = []

    for filename in os.listdir(folder_path):

        if filename.endswith(".jpg"):  # Assuming your images are in JPG format

            image_path = os.path.join(folder_path, filename)

            image = cv2.imread(image_path)

            image = cv2.resize(image, target_size)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

            image = histogram_equalization(image)

            image = image.astype('float32') / 255.0  # Normalize

            image_text = " ".join(map(str, image.flatten()))  # Convert image to a space-separated string

            images_text.append(image_text)



    labels = [label] * len(images_text)



    return images_text, labels



# Load and preprocess images from 'bad form' folder and convert them into text representations

bad_form_images_text, bad_form_labels = load_and_preprocess_images_to_text(r'C:/Users/Sejal/Downloads/project1_files/fip/bad form', label=0)





# Load and preprocess images from 'good form' folder and convert them into text representations

good_form_images_text, good_form_labels = load_and_preprocess_images_to_text(r'C:/Users/Sejal/Downloads/project1_files/fip/good form', label=1)





# Combine the datasets and labels

images_text = bad_form_images_text + good_form_images_text

labels = bad_form_labels + good_form_labels



# Split the combined dataset into training, validation, and test sets

X_train, X_temp, y_train, y_temp = train_test_split(images_text, labels, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



# Create a CountVectorizer to convert the text data into a bag of words representation

vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(X_train)

X_val = vectorizer.transform(X_val)

X_test = vectorizer.transform(X_test)



# Initialize and train the Multinomial Naive Bayes classifier

clf = MultinomialNB()

clf.fit(X_train, y_train)



# Make predictions on the validation set

y_val_pred = clf.predict(X_val)



# # Calculate the validation accuracy

# validation_accuracy = accuracy_score(y_val, y_val_pred)

# print(f'Validation Accuracy: {validation_accuracy * 100:.2f}%')



# Make predictions on the test set

y_test_pred = clf.predict(X_test)



# Calculate the test accuracy

report = classification_report(y_test, y_test_pred)

print(f'Classification report: {report}')



# # Save the trained classifier and vectorizer to a pickle file

# joblib.dump(clf, 'naive_bayes_classifier.pkl')

# joblib.dump(vectorizer, 'count_vectorizer.pkl')
